In [13]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
"""
ChatOllama 支援聊天提示和多輪對話，比較適合多輪交互；
而 OllamaLLM 主要用於單輸入單輸出，較簡單的應用。
語法和功能不同，ChatOllama 比較適合現在這種聊天場景。
"""
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7
)


In [14]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_name}"),
    ]
)

In [15]:
pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
    )

# 使用 LCEL 簡化分支鏈的建立
# 優點分析分支
pros_branch_chain = (
    pros_template 
    | model 
    | StrOutputParser()
)

In [16]:
cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
    )

# 缺點分析分支
cons_branch_chain = (
    cons_template
    | model
    | StrOutputParser()
)

In [17]:
# 定義結果合併函數
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

print("結果合併函數已定義：combine_pros_cons")

結果合併函數已定義：combine_pros_cons


In [18]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x:combine_pros_cons(x['pros'],x['cons']))
)

In [19]:
# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_name": restaurant_info})

print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
好的，請提供餐廳資訊，我會根據這些資訊來分析餐廳的優點。

假設餐廳名稱是 **“La Piazza”**，菜單價格範圍如下：

*   前菜：NT$400-600
*   義大利麵：NT$700-900
*   披薩：NT$800-1000
*   海鮮料理：NT$1000-1500
*   甜點：NT$300-500

用餐氛圍：輕鬆、浪漫，搭配輕音樂和昏黃燈光，適合情侶約會或朋友聚餐。

特殊特色：La Piazza 使用台灣在地的新鮮海鮮，並提供季節限定的特色菜色。

**根據以上資訊，La Piazza 的優點分析如下：**

**優點：**

*   **高品質的食材：** La Piazza 強調使用台灣在地的新鮮海鮮，這在台北市中心提供義式料理的餐廳中是顯著的優勢。這種強調本地食材的策略，能讓顧客感受到食材的新鮮度和品質，提升整體用餐體驗。
*   **浪漫的用餐氛圍：** 餐廳的裝潢和佈置，搭配輕音樂和昏黃燈光，營造出輕鬆浪漫的用餐氛圍，非常適合情侶約會或朋友聚餐，滿足顧客對特殊場合用餐的期待。
*   **精緻的菜色選擇：** 菜單提供多樣化的選擇，包含經典的義大利麵、披薩，以及以台灣海鮮為主題的特色料理，滿足不同顧客的口味偏好。
*   **強調季節限定菜色：** 採用季節限定菜色，能帶來新鮮感和話題性，也讓顧客有機會品嚐到獨一無二的美味，增加用餐的樂趣。
*   **位於市中心便利地點：** 餐廳位於市中心，交通便利，方便顧客前往。

總體而言，La Piazza 在食材、環境、菜色選擇和便利的地點方面都表現出色，為顧客提供了一次優質的義式料理體驗。


❌ 缺點分析：
好的，我來提供一些資訊，以便更精準地分析餐廳的缺點。

**餐廳名稱：** La Piazza (意為「廣場」)

**菜單價格範圍：**

*   前菜 (Antipasto)：NT$480 - NT$780
*   義大利麵 (Pasta)：NT$780 - NT$1280
*   披薩 (Pizza)：NT$880 - NT$1480
*   海鮮料理 (Pesce)：NT$1280 - NT$2280
*   甜點 (Dolci)：NT$380 - NT$680

**主要菜色：**

*   經典義式披薩 (Margher